# Library used

In [1]:
## Library 
import pandas as pd
import numpy as np
import scipy as scipy
from scipy import spatial
import json
import argparse
from argparse import Namespace
import glob
import cv2 as cv2
import sklearn as sk
from sklearn import model_selection
from sklearn import metrics
from scipy.ndimage import gaussian_filter1d
import matplotlib.pyplot as plt 
import matplotlib.animation as animation
from matplotlib.animation import PillowWriter
from matplotlib import cm

import time
import os
import datetime

import tensorflow as tf
import tensorflow.keras as keras

import torch
import torch.nn as nn
import torch.nn.functional as F

## Utils function from https://github.com/ChrisWu1997/2D-Motion-Retargeting
from functional.visualization import motion2video, hex2rgb
from functional.motion import preprocess_motion2d, postprocess_motion2d, openpose2motion
from functional.utils import ensure_dir, pad_to_height
from model import get_autoencoder

from common import config
from dataset import get_meanpose

# Some global variables

In [2]:
## Indicating the joint connection
connect_dict=[[0,1],[1,2],[2,3],[3,4],[4,5],[3,6],[6,7],[7,8],[8,9],[3,10],[10,11],[11,12],[12,13],[0,14],[14,15],[15,16],[16,17],[0,18],[18,19],[19,20],[20,21]]
pose_net_joint=[4,7,8,9,11,12,13,14,15,16,18,19,20]
open_pose_joint=[4,2,11,12,13,7,8,9,0,18,19,20,14,15,16]
## Sliding window size
win_size=16
## Task and participant name
task_list=['m01','m02','m03','m04','m05','m06','m07','m08','m09','m10']
s_number=['s01','s02','s03','s04','s05','s06','s07','s08','s09','s10']
e_number=['e01','e02','e03','e04','e05','e06','e07','e08','e09','e10']
name_dict={'m01':'deep squat','m02':'hurdle step','m03':'inline lunge','m04':'side lunge','m05':'sit to stand','m06':'leg raise','m07':	'shoulder abduction','m08':	'shoulder extension','m09':	'shoulder internal-external rotation','m10':'shoulder scaption'}

## Mirroring the motion if the participants are doing in the minority side
LR_table=pd.read_csv('LR_table.csv',index_col=0)
## Kinect position/angle data location
kin_position_path='Segmented Movements/Segmented Movements/Kinect/Positions/'
kin_angle_path='Segmented Movements/Segmented Movements/Kinect/Angles'

# Utility functions

In [3]:
def to_3D_corrd_in_view(kinect_postion,kinect_angle):
    kinect_postion=np.array(kinect_postion)
    kinect_angle=np.array(kinect_angle)
    kinect_postion=kinect_postion.reshape(22,3)
    kinect_angle=kinect_angle.reshape(22,3)
    for j_p, j_c in connect_dict:
        rot_mat=scipy.spatial.transform.Rotation.from_euler('yxz',kinect_angle[j_p,:3]*np.pi/180).as_matrix()
        kinect_postion[j_c,:]=kinect_postion[j_p,:]+np.matmul(rot_mat,kinect_postion[j_c,:])   
    return kinect_postion

def plot_skelton(dat,ax,**kwargs):
    ax.invert_xaxis()
    ax.invert_yaxis()
    for j1,j2 in connect_dict:
        ax.plot(dat[[j1,j2],0],-dat[[j1,j2],1],**kwargs)
    return ax
        
def LR_mirror(kinect_postion,percentage,LR_table,motion): 
    for i in range(len(percentage)):
        if LR_table[motion][percentage.tester[i]]:
            kinect_postion[i,:,0]=-kinect_postion[i,:,0]
    return kinect_postion

def poseNet2openPose(kinect_postion):
    chest=np.mean(kinect_postion[:,[1,4],:],axis=1).reshape(-1,1,kinect_postion.shape[2])
    base=np.mean(kinect_postion[:,[7,10],:],axis=1).reshape(-1,1,kinect_postion.shape[2])
    new_openpose=kinect_postion[:,[0],:]

    new_openpose=np.concatenate((new_openpose,chest),axis=1)
    new_openpose=np.concatenate((new_openpose,kinect_postion[:,[4,5,6],:]),axis=1)
    new_openpose=np.concatenate((new_openpose,kinect_postion[:,[1,2,3],:]),axis=1)
    new_openpose=np.concatenate((new_openpose,base),axis=1)
    new_openpose=np.concatenate((new_openpose,kinect_postion[:,[10,11,12],:]),axis=1)
    new_openpose=np.concatenate((new_openpose,kinect_postion[:,[7,8,9],:]),axis=1)
    return new_openpose

def getPoseNet(motion):
    return(motion[:,pose_net_joint,:])
def getOpenPose(motion):
    return(motion[:,open_pose_joint,:])
def to_2D(motion):
    return(motion[:,:,:2])

def tl_preprocess_encode(motion,encoder,scale=1.2):
    
    motion=to_2D(motion)

    if motion.shape[1] == 22:
        motion=getOpenPose(motion)
    elif motion.shape[1] == 13:
        motion=poseNet2openPose(motion)
    elif motion.shape[1] == 15:
        motion=motion
    else:
        "Not supported skeleton"
    for i in range(len(motion) - 1, 0, -1):
        motion[i - 1][np.where(motion[i - 1] == 0)] = motion[i][np.where(motion[i - 1] == 0)]

    motion = np.stack(motion, axis=2)
    motion = gaussian_filter1d(motion, sigma=2, axis=-1)
    motion = motion * scale
    
    motion=preprocess_motion2d(motion, mean_pose, std_pose)
    motion=motion.to(config.device)
    return encoder(motion).cpu().detach().numpy().squeeze()

def get_sliding_wins(motion,percentage,win_size,steps=1):
    percentage.reset_index(inplace=True,drop=True)
    sliding_win=[]
    sliding_percent=[]

    for i in percentage.tester.unique():
        start_id=np.where(percentage.tester == i)[0][0]
        end_id=np.where(percentage.tester == i)[0][-1]+1
        counter=0
        for window in percentage.percentage.iloc[start_id:end_id].rolling(window=win_size,min_periods=win_size):
            if len(window)<win_size:
                continue
            else:
                if counter%steps == 0:
                    sliding_win.append(motion[window.index,:,:])
                    sliding_percent.append(window.iloc[-1])
                    counter+=1
                else:
                    counter+=1
    sliding_percent=np.array(sliding_percent)
    return(sliding_win,sliding_percent)

def label_encoding(data, segment,win_size,encoder,steps=1):
    sliding_win, sliding_percent=get_sliding_wins(data, segment,win_size,steps=steps)
    encode_test=[]
    for i in range(len(sliding_win)):
        data=tl_preprocess_encode(sliding_win[i],encoder=encoder)
        encode_test.append(data)
    encode_test=np.array(encode_test)
    # sim_matrix=sk.metrics.pairwise.cosine_similarity(encode_test.reshape((-1,96*3)))
    return((encode_test, sliding_percent) )

# Create the encoder

In [4]:
args= argparse.Namespace(name='skeleton',model_path='model/pretrained_skeleton.pth',v1='inc_deep_squat',o='inc_deep_squat',gpu_ids=0,w1=720,h1=720,transparency=False,save_frame=1,
 fps=25,color1='#a50b69#b73b87#db9dc3',max_len=480,max_frame=480)
config.initialize(args)
mean_pose, std_pose = get_meanpose(config)
net = get_autoencoder(config)
net.load_state_dict(torch.load(args.model_path))
net.to(config.device)
net.eval()
encoder=nn.Sequential(*list(net.mot_encoder.children())[0])

# Training model

In [5]:
for task in task_list:
    if task != 'm10':
        continue
    m_data=np.load('data/'+task+'.npy')
    percentage=pd.read_csv('data/'+task+'.csv')
    m_data=LR_mirror(m_data,percentage,LR_table,task)
    m_data=getPoseNet(m_data)
    for subj in s_number:
        print("Motion: %s"%task)
        print('Testing on subject: %s'%subj)
        X_train, y_train = np.array(m_data)[np.where(percentage['tester']!=subj),:,:][0],percentage.iloc[np.where(percentage['tester']!=subj)[0],:]
        X_test, y_test = np.array(m_data)[np.where(percentage['tester']==subj),:,:][0],percentage.iloc[np.where(percentage['tester']==subj)]




        encode_X_train_camera, sim_matrix_X_train_camera =  label_encoding(X_train, y_train,win_size,encoder,steps=1)


        print("Finish encoding")

        X_train, y_train =sk.utils.shuffle(encode_X_train_camera,sim_matrix_X_train_camera)


        print(X_train.shape)
        print(y_train.shape)

        model = keras.models.Sequential()
        model.add(keras.Input(shape=(128,2,)))
        model.add(keras.layers.BatchNormalization())

        model.add(keras.layers.Conv1D(32, 3, activation='relu'))
        model.add(keras.layers.MaxPooling1D(2))
        model.add(keras.layers.Conv1D(64, 3, activation='relu'))
        model.add(keras.layers.MaxPooling1D(2))
        model.add(keras.layers.Flatten())

        model.add(keras.layers.Dense(16))
        model.add(keras.layers.Dense(8))
        model.add(keras.layers.Dense(4))
        model.add(keras.layers.Dense(1,activation='sigmoid'))
        model.compile(
        optimizer=keras.optimizers.Adam(),
        loss="mean_squared_error",
        metrics=['MeanSquaredError']
        )
        print(model.summary())
        model.fit(x=X_train.astype('float32'),y=y_train.astype('float32'), epochs=1000,validation_split=0.2,batch_size=50)
        model.save('model/0516_full_full_Motion_sim_poseNet_'+task+"_testing on_"+subj+'.h5')

        X_testcamera=np.array([camera_projet(x,np.array([0,0,0.0]),np.array([0,0.0,0])) for x in X_test])
        encode_X_test_camera, sim_matrix_X_test_camera =  label_encoding(X_testcamera, y_test,win_size,encoder)

        predict=model.predict(encode_X_test_camera)

Motion: m10
Testing on subject: s01
Finish encoding
(5752, 128, 2)
(5752,)
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo (None, 128, 2)            8         
_________________________________________________________________
conv1d (Conv1D)              (None, 126, 32)           224       
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 63, 32)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 61, 64)            6208      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 30, 64)            0         
_________________________________________________________________
flatten (Flatten)            (None, 1920)              0         
_______________________________________________

KeyboardInterrupt: 

# Counting

In [ ]:

lb=np.arange(0.1,1,0.05)
bound=np.arange(0.2,1,0.05)
for task in task_list:
    m_data=np.load('data/'+task+'.npy')
    percentage=pd.read_csv('data/'+task+'.csv')
    m_data=LR_mirror(m_data,percentage,LR_table,task)
    m_data=getOpenPose(m_data)
    space=pd.DataFrame(index=lb,columns=bound)
    tot_time=0
    tot_frame=0
    for subj in s_number:
        print("Motion: %s"%task)
        print('Testing on subject: %s'%subj)
        X_train, y_train = np.array(m_data)[np.where(percentage['tester']!=subj),:,:][0],percentage.iloc[np.where(percentage['tester']!=subj)[0],:]
        X_test, y_test = np.array(m_data)[np.where(percentage['tester']==subj),:,:][0],percentage.iloc[np.where(percentage['tester']==subj)]

        model=tf.keras.models.load_model('0516_model/0516_full_full_Motion_sim_poseNet_'+task+"_testing on_"+subj+'.h5')

        start=time.time()
        encode_X_test_camera, sim_matrix_X_test_camera =  label_encoding(X_test, y_test,win_size,encoder)
        predict=model.predict(encode_X_test_camera)
        end=time.time()
        perfor_t[task][subj]=(end-start)/encode_X_test_camera.shape[0]
        perfor_mse[task][subj]=sk.metrics.mean_squared_error(sim_matrix_X_test_camera,predict)
        frame_time= np.where(sim_matrix_X_test_camera==1)[0]
        frame_time=np.concatenate((frame_time,[len(sim_matrix_X_test_camera)]))
        previous_best=[]
        current_frame=[]
        for l in lb:
            for ran in bound:
                if l+ran >1:
                    continue
                count=0
                low_swtich=True
                high_switch=False
                current_frame=[]
                for i in range(len(predict)):
                    if predict[i]<l and low_swtich:
                        high_switch=True
                        low_swtich=False
                    if predict[i]>l+ran and high_switch:
                        count=count+1
                        high_switch=False
                        low_swtich=True
                        current_frame.append(i)
                if perfor[task][subj] is np.nan or (np.abs(count-10)<np.abs(perfor[task][subj]-10)) or (np.abs(count-10)==np.abs(perfor[task][subj]-10) and ran> perfor_r[task][subj]):       
                    perfor[task][subj]= count
                    perfor_lb[task][subj]=l
                    perfor_r[task][subj]=ran

        if len(previous_best) ==10:
            perfor_t_diff[task][subj]=np.mean(np.abs(np.array(previous_best)-frame_time)) 
        fig, ax= plt.subplots()
        ax.set_title("Motion: "+name_dict[task]+" testing on "+subj)
        ax.hlines([perfor_lb[task][subj],perfor_lb[task][subj]+perfor_r[task][subj]],0,len(predict))
        ax.annotate('Lower bound',(10,perfor_lb[task][subj]),xytext=(30,perfor_lb[task][subj]+0.1),arrowprops=dict(arrowstyle="->", connectionstyle="arc3,rad=0.2"))
        ax.annotate('Upper bound',(10,perfor_lb[task][subj]+perfor_r[task][subj]),xytext=(30,perfor_lb[task][subj]+perfor_r[task][subj]-0.1),arrowprops=dict(arrowstyle="->", connectionstyle="arc3,rad=0.2"))
        ax.plot(predict)
        ax.plot(sim_matrix_X_test_camera)
        ax.set_xlabel("Frame")
        ax.set_ylabel("Progress")
        ax.legend(['Predicted value','Groud Truth'])


# Making video

Select task and subject

In [36]:
motion='m05'
subject='s03'
output_file='demo3.gif'
lower=0.2
upper=0.8

In [37]:
m_data=np.load('data/'+motion+'.npy')
percentage=pd.read_csv('data/'+motion+'.csv')
m_data=LR_mirror(m_data,percentage,LR_table,motion)
X_test, y_test = np.array(m_data)[np.where(percentage['tester']==subject),:,:][0],percentage.iloc[np.where(percentage['tester']==subject)]
X_test_op=getOpenPose(X_test)
model=tf.keras.models.load_model('0516_model/0516_full_full_Motion_sim_poseNet_'+motion+"_testing on_"+subject+'.h5')
encode_X_test_camera, sim_matrix_X_test_camera =  label_encoding(X_test_op, y_test,win_size,encoder)
predict=model.predict(encode_X_test_camera)

In [38]:
fig, ax= plt.subplots()
count=0
low_swtich=True
high_switch=False
ims = []
for i in range(len(X_test)):
    ## Plot skeleton
    ske=[]
    for j1,j2 in connect_dict:
        line,=ax.plot(X_test[i][[j1,j2],0],X_test[i][[j1,j2],1],c="#81b941")
        ske.append(line)
    ## Counting
    if i-16 > 0 and i-16<len(predict): 
        if predict[i-16]<lower and low_swtich:
            high_switch=True
            low_swtich=False
        if predict[i-16]>upper and high_switch:
            count=count+1
            high_switch=False
            low_swtich=True
            print("Count %s at frame %s"%(count,i))
    p_text=ax.text(-60,60,"Predict: %.2f"%predict[i-16])
    c_text=ax.text(-60,40,"Count: %s"%count)
    ske.append(p_text)
    ske.append(c_text)
    ims.append(ske)
    plt.close()

Count 1 at frame 90
Count 2 at frame 190
Count 3 at frame 288
Count 4 at frame 391
Count 5 at frame 496
Count 6 at frame 602
Count 7 at frame 728
Count 8 at frame 843
Count 9 at frame 951
Count 10 at frame 1057


In [39]:

ani = animation.ArtistAnimation(fig, ims, interval=33, blit=True,
                                repeat_delay=1)

writer = PillowWriter(fps=33)
ani.save(output_file, writer=writer)